**GEOSPATIAL DATA**

In [1]:
from geopy.geocoders import Nominatim

In [3]:
geolocator = Nominatim(user_agent='kaggle_learn')
location = geolocator.geocode('Pyramid of Khufu')
print(location.point)
print(location.address)

29 58m 44.9927s N, 31 8m 3.16774s E
هرم خوفو, شارع ابو الهول السياحي, كوم الأخضر, الجيزة, 12125, مصر


In [4]:
import pandas as pd


universities = pd.read_csv('top_universities.csv')
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [5]:
import numpy as np
import geopandas as gpd

def my_geocoder(row):
  try:
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
  except:
    print('An exception occurred')
    return None
universities[['Latitude', 'Longitude']] = universities.apply(
  lambda x: my_geocoder(x['Name']), axis=1
)
universities = universities.loc[~np.isnan(universities['Latitude'])]
universities = gpd.GeoDataFrame(universities, geometry=gpd.points_from_xy(
  universities.Longitude, universities.Latitude
))
universities.crs = {'init': 'epsg:4326'}
universities.head()

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


/Users/asaucedov/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.210946,0.092005,POINT (0.09200 52.21095)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179)


In [6]:
import folium

m = folium.Map(localtion=[54, 15], tiles='openstreetmap', zoom_start=2)
for idx, row in universities.iterrows():
  folium.Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)
m